# Apriori Algoritması ile Film Öneri Sistemi
Kullanıcıların izleme alışkanlıklarına dayalı olarak öneriler sunmak için MovieLens 20M veri setini kullanarak birliktelik kuralları temelinde bir öneri sistemi oluşturulmuştur. Apriori algoritması, sıklıkla izlenen film kombinasyonlarını ve bunlara bağlı önerileri belirlemeye yardımcı olur.
**Proje Adımları**:
- Veri Yükleme ve Temizleme
- Kullanıcı-Film Matrisi Oluşturma
- Apriori Algoritmasının Uygulanması
- Birliktelik Kurallarının Çıkarılması
- Performans Analizi

In [ ]:
# kütüphaneleri import et
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

# setleme 
pd.set_option('display.max_columns', None)


## 1. Veri Yükleme ve Keşif
MovieLens veri seti çekildi ve veriler açıldı.

In [ ]:
# verileri yükle
ratings = pd.read_csv(r"C:\\Users\\furkn\\Desktop\\archive\\dosya.csv")  
movies = pd.read_csv(r"C:\\Users\\furkn\\Desktop\\archive\\movie.csv")    

# veri setlerini oku
print('Ratings Data:')
print(ratings.head())
print('\nMovies Data:')
print(movies.head())

Ratings Data:
   userId  movieId  rating            timestamp
0       1      151     4.0  2004-09-10 03:08:54
1       1      541     4.0  2005-04-02 23:30:03
2       1     1097     4.0  2005-04-02 23:48:21
3       1     1136     3.5  2005-04-02 23:30:09
4       1     1214     4.0  2004-09-10 03:12:57

Movies Data:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


## 2. Veri Temizleme
Gereksiz veriler veri setinden temizlendi.

In [ ]:
# kayıp verileri al
print(ratings.isnull().sum())
print(movies.isnull().sum())

# gereksizleri gönder
ratings.dropna(inplace=True)
movies.dropna(inplace=True)

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64
movieId    0
title      0
genres     0
dtype: int64


## 3. Veri Dönüşümü - Kullanıcı-Film Matrisi
İzlenen filmleri 1, izlenmeyen filmleri 0 olarak gösteren bir kullanıcı-film matrisi hazırlandı.

In [ ]:
# filtreleme
high_ratings = ratings[ratings['rating'] >= 4]

# matris
user_movie_matrix = high_ratings.pivot(index='userId', columns='movieId', values='rating')
user_movie_matrix = user_movie_matrix.notna().astype(int)
user_movie_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,129,130,131,132,134,135,136,137,138,140,141,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,404,405,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,431,432,433,434,435,436,437,438,440,441,442,443,444,445,446,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,558,559,561,562,563,564,565,567,568,569,570,571,573,574,575,577,580,581,582,583,584,585,586,587,588,589,590,592,593,594,595,596,597,598,599,600,601,602,603,605,606,608,609,610,611,612,613,614,615,616,617,618,619,621,626,627,628,630,631,632,633,634,635,636,637,638,639,640,645,647,648,649,650,651,653,656,659,660,661,662,663,664,665,666,667,668,670,671,673,674,676,678,679,680,681,682,685,688,691,692,694,695,696,697,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,722,724,725,726,728,729,731,732,733,735,736,737,741,742,743,745,746,747,748,750,751,752,753,754,755,756,757,759,760,761,762,764,765,766,767,769,771,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,812,813,818,819,821,823,824,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,844,845,846,847,848,849,850,851,852,853,854,857,858,860,861,862,864,866,867,868,869,870,871,872,873,874,875,876,877,879,880,881,882,885,886,888,889,890,891,892,893,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,973,974,975,976,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,996,997,998,999,1000,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1040,1041,1042,1043,1044,1046,1047,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1066,1067,1068,1069,1071,1073,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1103,1104,1105,1111,1112,1113,1114,1115,1116,1117,1119,1120,1123,1124,

## 4. Türe Göre Popüler Filmler
İzlenme ve oylanma sayısına göre her türde en popüler 50 film belirlendi.

In [ ]:
# birleştir
merged_df = pd.merge(high_ratings, movies, on='movieId')

# popüler 50 film
popular_movies = merged_df.groupby(['genres', 'movieId'])['userId'].count().reset_index()
popular_movies.columns = ['genre', 'movieId', 'view_count']
top_50_per_genre = popular_movies.groupby('genre').apply(lambda x: x.nlargest(50, 'view_count'))
top_50_per_genre.tail()

C:\Users\furkn\AppData\Local\Temp\ipykernel_3248\3900054306.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_50_per_genre = popular_movies.groupby('genre').apply(lambda x: x.nlargest(50, 'view_count'))


genre  movieId  view_count
genre                                      
Western 11315  Western     2896           1
        11322  Western     3642           1
        11331  Western     4790           1
        11332  Western     4791           1
        11334  Western     5149           1

## 5. Apriori Algoritmasının Uygulanması
Kullanıcı-film matrisinden sık öğe kümeleri çıkarıldı.

In [ ]:
#apriori uygula
frequent_itemsets = apriori(user_movie_matrix, min_support=0.4, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False).head()

c:\Users\furkn\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets


In [ ]:

all_genres = set([genre for sublist in movies['genres'].str.split('|') for genre in sublist])
print("Tüm türler:", all_genres)


for genre in all_genres:
    movies[genre] = movies['genres'].apply(lambda x: 1 if genre in x else 0)


print(movies.head())


Tüm türler: {'Adventure', 'Sci-Fi', 'Western', 'Horror', 'Comedy', 'Mystery', 'War', 'Romance', '(no genres listed)', 'Drama', 'Children', 'Fantasy', 'Animation', 'Crime', 'Documentary', 'IMAX', 'Thriller', 'Musical', 'Film-Noir', 'Action'}
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  Adventure  Sci-Fi  Western  \
0  Adventure|Animation|Children|Comedy|Fantasy          1       0        0   
1                   Adventure|Children|Fantasy          1       0        0   
2                               Comedy|Romance          0       0        0   
3                         Comedy|Drama|Romance          0       0        0   
4                                       Comedy          0       0   

## 6. Birliktelik Kurallarının Çıkarılması
Destek,güven ve kaldıraç metriklerine göre sık öğe kümelerine dayalı kurallar oluşturuldu.

In [ ]:
# birliktelik kuralı sınırı
frequent_itemsets = apriori(user_movie_matrix, min_support=0.02, use_colnames=True)
print('Frequent itemsets:', frequent_itemsets)

#df boşmu kontrol et
if not frequent_itemsets.empty:
    
    rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.6)
    rules = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
    display(rules.head())
else:
    print('No frequent itemsets found. Try reducing the minimum support threshold.')

c:\Users\furkn\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Frequent itemsets:     support itemsets
0  0.023485    (296)
1  0.025485    (318)
2  0.021808    (356)
3  0.022197    (593)


,antecedents,consequents,support,confidence,lift


In [ ]:

rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.6)
rules = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
rules.head()

,antecedents,consequents,support,confidence,lift


## 7. Performans Analizi
Üçgen Ağaç veri yapısı kullanıldı ve kullanmadan öğe kümelerini depolamanın bellek ve hesaplama performansı incelendi.

In [ ]:
# performans analizi
import time

start_time = time.time()
# üçgen ağaç 
print('Execution time:', time.time() - start_time)

Execution time: 0.0008006095886230469


In [ ]:

movies['genres'] = movies['genres'].str.split('|')


exploded_movies = movies.explode('genres')


merged_df = pd.merge(high_ratings, exploded_movies, on='movieId')

popular_movies_by_genre = merged_df.groupby(['genres', 'movieId'])['userId'].count().reset_index()
popular_movies_by_genre.columns = ['genre', 'movieId', 'view_count']


top_50_per_genre = popular_movies_by_genre.groupby('genre').apply(lambda x: x.nlargest(50, 'view_count')).reset_index(drop=True)


print(top_50_per_genre.head())


                genre  movieId  view_count
0  (no genres listed)    94431           1
1  (no genres listed)    95541           1
2  (no genres listed)   117192           1
3  (no genres listed)   126106           1
4  (no genres listed)   126955           1


C:\Users\furkn\AppData\Local\Temp\ipykernel_3248\535144987.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_50_per_genre = popular_movies_by_genre.groupby('genre').apply(lambda x: x.nlargest(50, 'view_count')).reset_index(drop=True)
